### 21.07.25 Team Complete

# lightgbm

In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error
from lightgbm import LGBMRegressor

In [2]:
seed =2021
np.random.seed(seed)

In [3]:
# 01. 데이터불러오기
df_감말랭이_raw = pd.read_csv('감말랭이.csv', encoding='cp949')
df_감말랭이 = df_감말랭이_raw #원본저장

In [4]:
df_감말랭이 = df_감말랭이.set_index('date') #date를 index로 지정

## Target data 스케일링 

In [5]:
# 02. 전처리
df_감말랭이['감말랭이_scaling'] = MinMaxScaler().fit_transform(df_감말랭이[['감말랭이']]) # 감말랭이열 scaling
df_감말랭이

,PC1,PC2,감말랭이,감말랭이_scaling
date,,,,
2018-01-01T00:00:00Z,-3.719474,0.965108,2.862813,0.252020
2018-01-02T00:00:00Z,-3.741912,0.888377,4.328310,0.385424
2018-01-03T00:00:00Z,-4.074829,0.748572,5.545913,0.496263
2018-01-04T00:00:00Z,-4.325838,0.235772,5.867105,0.525501
2018-01-05T00:00:00Z,-3.772842,0.246325,4.414783,0.393296
...,...,...,...,...
2019-12-27T00:00:00Z,-3.280702,1.070884,4.136090,0.367926
2019-12-28T00:00:00Z,-3.036702,0.091263,2.425077,0.212172
2019-12-29T00:00:00Z,-2.978009,-1.269522,2.476555,0.216858


In [6]:
del df_감말랭이['감말랭이'] # 스케일링 이전 column delete
df_감말랭이

,PC1,PC2,감말랭이_scaling
date,,,
2018-01-01T00:00:00Z,-3.719474,0.965108,0.252020
2018-01-02T00:00:00Z,-3.741912,0.888377,0.385424
2018-01-03T00:00:00Z,-4.074829,0.748572,0.496263
2018-01-04T00:00:00Z,-4.325838,0.235772,0.525501
2018-01-05T00:00:00Z,-3.772842,0.246325,0.393296
...,...,...,...
2019-12-27T00:00:00Z,-3.280702,1.070884,0.367926
2019-12-28T00:00:00Z,-3.036702,0.091263,0.212172
2019-12-29T00:00:00Z,-2.978009,-1.269522,0.216858


## train/ test set split

In [7]:
#03. train/ test 분리
X_train, X_test, y_train, y_test = df_감말랭이.iloc[0:365,:-1], df_감말랭이.iloc[365:730,:-1],df_감말랭이.iloc[0:365,-1], df_감말랭이.iloc[365:730,-1]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((365, 2), (365,), (365, 2), (365,))

## 모델 생성, 학습 및 예측, 평가

In [8]:
#04. 모델 생성 및 학습
lgb_rg = LGBMRegressor()

# 하이퍼 파라미터 확인
lgb_rg.get_params() 

{'boosting_type': 'gbdt',
 'class_weight': None,
 'colsample_bytree': 1.0,
 'importance_type': 'split',
 'learning_rate': 0.1,
 'max_depth': -1,
 'min_child_samples': 20,
 'min_child_weight': 0.001,
 'min_split_gain': 0.0,
 'n_estimators': 100,
 'n_jobs': -1,
 'num_leaves': 31,
 'objective': None,
 'random_state': None,
 'reg_alpha': 0.0,
 'reg_lambda': 0.0,
 'silent': True,
 'subsample': 1.0,
 'subsample_for_bin': 200000,
 'subsample_freq': 0}

In [9]:
# gridsearchCV
params = {
    'max_depth': [7,8,9],
    'min_child_samples': [22,23,24,25,26,27],
    'learning_rate' : [0.01, 0.02, 0.03],
    'n_estimators' : [497,498,499,500,501,502,503] 
}
gscv_lgb = GridSearchCV(lgb_rg, param_grid=params, cv=10)

In [10]:
# 모형학습
gscv_lgb.fit(X_train,y_train, eval_metric='logloss') 

# 최적파라미터 확인
gscv_lgb.best_params_

{'learning_rate': 0.01,
 'max_depth': 7,
 'min_child_samples': 25,
 'n_estimators': 497}

In [11]:
# 최적파라미터에 대한 평가점수 확인
gscv_lgb.best_score_

-9.634068937962962

In [12]:
#05. 평가
# test 데이터셋으로 모형 예측 및 평가

# R^2
gscv_xgb_fit = gscv_lgb.best_estimator_
lgb_pred = gscv_xgb_fit.predict(X_test)
r2_score(y_test,lgb_pred)

0.2589829413941712

In [13]:
# RMSE
mse = mean_squared_error(y_test,lgb_pred)
rmse = np.sqrt(mse)
rmse

0.13855507333196618